In [17]:
from hpacker import HPacker

# Quick Start - right to packing

## 1) Bare Backbone + Sequence information

In [18]:
# Initialize HPacker object by passing it a tutple of paths to the pre-trained models,
# and the backbone-only structure that we want to add side-chains to.
hpacker = HPacker('T0950_bb_only.pdb')

There are **three** equivalent ways of telling HPacker which amino-acid should correspond to which site.\
The **first** - and simplest - way is to have the input PDB file already contain sequence information (as is the case in our working example).
All we need to do then is a simple call to `.reconstruct_sidechains()`.

In [19]:
chi_trace = hpacker.reconstruct_sidechains(num_refinement_iterations=5, return_trace_of_predicted_angles=True)

# the reconstructed structure will be saved internally in `hpacker.structure`, and can be easiuly saved in a PDB file
hpacker.write_pdb('reconstructed_from_bb_only_T0950.pdb')

All sidechains are missing in the structure. Using the initial guess model.


In [20]:
# `chi_trace` contains a record of the predicted chi angles across refinement iterations. it is a list of dictionaries
# within each dictionary, chi angles for individual residues are indexed by residue IDs (res_id), which are Tuples of (chain, resnum, icode)
initial_guess_chis = chi_trace[0]
final_chis = chi_trace[-1]

res_ids = hpacker.get_res_ids()
for i in range(10):
    res_id = res_ids[i]
    print(f'{res_id}: {final_chis[res_id]}')

(' ', 12, ' '): [178.8607940673828]
(' ', 13, ' '): [-46.83998489379883]
(' ', 14, ' '): [-53.04864501953125, -32.65345001220703]
(' ', 15, ' '): [19.594274520874023, -25.412826538085938]
(' ', 16, ' '): [-157.41934204101562, 176.36685180664062, 175.0377960205078]
(' ', 17, ' '): [-57.67374038696289, 170.25657653808594]
(' ', 18, ' '): [-174.62457275390625, -20.914920806884766]
(' ', 19, ' '): [-60.83747100830078]
(' ', 20, ' '): [-70.23597717285156, -179.6685791015625, 176.46070861816406, 179.88401794433594]
(' ', 21, ' '): [-63.55720520019531]


The **second** way of introducing sequence information is to tell HPacker via the `.update_resnames()` method, before reconstructing.

In [21]:
# res_id_to_resname = {} # dictionary: res_id --> three-letter amino-acid code
# hpacker.update_resnames(res_id_to_resname)

**Third**, the same information can be passed directly into the reconstruction function.

In [22]:
# hpacker.reconstruct_sidechains(num_refinement_iterations=5, res_id_to_resname=res_id_to_resname)

## 2) Repacking an All-Atom structure

In [23]:
# when passing as input a structure that already has side-chains and that you want to re-pack from zero,
# you should toggle the `remove_sidechains` flag
hpacker = HPacker('T0950.pdb', remove_sidechains=True)

In [24]:
# as sequence information is already available, repacking the side-chains is a one-liner
hpacker.reconstruct_sidechains(num_refinement_iterations = 5)
hpacker.write_pdb('reconstructed_from_AA_v1_T0950.pdb')

All sidechains are missing in the structure. Using the initial guess model.


In [25]:
# alternatively, you can just set `reconstruct_all_sidechains` to `True` when reconstructing
hpacker = HPacker('T0950.pdb')
hpacker.reconstruct_sidechains(num_refinement_iterations = 5, reconstruct_all_sidechains=True)
hpacker.write_pdb('reconstructed_from_AA_v2_T0950.pdb')

Reconstructing all sidechains from scratch


#### If you want to evaluate against ground truth...

We provide a custom function that assumes the input structure is the ground truth, and returns several metrics comparing the reconstructed side-chains with the original ones

In [26]:
hpacker = HPacker('T0950.pdb', remove_sidechains=True)

metrics = hpacker.reconstruct_sidechains_and_evaluate(num_refinement_iterations = 5)

mae_per_angle_4, accuracy_per_angle_4, real_chis, predicted_chis, aas, res_ids_dict, rmsds = metrics

print(mae_per_angle_4)
print(rmsds.keys())
res_id_to_rmsd = dict(zip(res_ids_dict['all'], rmsds['all']))
for i in range(10):
    print('%s: %.3f Angstroms' % (res_ids_dict['all'][i], rmsds['all'][i]))

All residues: 309
Core residues: 79
Surface residues: 147
Accuracy:	76	66	46	62	
MAE:	19	29	41	36	

All residues: 309
Core residues: 79
Surface residues: 147
Accuracy:	77	70	47	62	
MAE:	19	29	39	36	

All residues: 309
Core residues: 79
Surface residues: 147
Accuracy:	76	69	49	66	
MAE:	18	27	38	34	

All residues: 309
Core residues: 79
Surface residues: 147
Accuracy:	77	71	48	66	
MAE:	18	27	38	36	

All residues: 309
Core residues: 79
Surface residues: 147
Accuracy:	76	69	49	64	
MAE:	19	28	37	38	

All residues: 309
Core residues: 79
Surface residues: 147
Accuracy:	77	70	47	66	
MAE:	18	27	38	36	

tensor([18.4906, 27.3550, 38.4248, 36.0792], dtype=torch.float64)
dict_keys(['all', 'core', 'surface'])
(' ', 12, ' '): 0.107 Angstroms
(' ', 13, ' '): 1.928 Angstroms
(' ', 14, ' '): 0.298 Angstroms
(' ', 15, ' '): 0.497 Angstroms
(' ', 16, ' '): 2.035 Angstroms
(' ', 17, ' '): 0.182 Angstroms
(' ', 18, ' '): 0.294 Angstroms
(' ', 19, ' '): 0.181 Angstroms
(' ', 20, ' '): 2.930 Angstroms
(' ', 21

#### If you only want to refine existing side-chains...

If you believe your model already has resonably good side-chains, and you just want to refine them, you can use Hpacker's refinement-only option. This only runs the refinement model on the existing side-chains, so you should expect the resulting side-chains to deviate less from the input structure then when making HPacker start from zero, as done above.

In [27]:
hpacker = HPacker('T0950.pdb')

# refinement only
hpacker.refine_sidechains(num_refinement_iterations = 5,
                          return_trace_of_predicted_angles = True)

hpacker.write_pdb('refined_only_T0950.pdb')

You can also specify which side-chains to refine, by passing a list of residue ids to the `res_ids` argument, where each residue id has the form `(chain, resnum, icode)`.

In [28]:
hpacker = HPacker('T0950.pdb')

res_ids_to_refine = hpacker.get_res_ids()[:10]
hpacker.refine_sidechains(res_ids = res_ids_to_refine,
                          num_refinement_iterations = 5,
                          return_trace_of_predicted_angles = True)

hpacker.write_pdb('refined_only_some_sidechains_T0950.pdb')

## 3) Inpainting: packing just part of a structure

HPacker supports inpaining, i.e. the packing of side-chains of only a *subset* of residues. This is particularly useful for applying mutations, and for designing interfaces.

Below we outline how to set the parameters of `.refine_sidechains()` to cover different use cases.

Let us first consider the case in which our input protein is missing some side-chains, and we want to add them.

In [29]:
# create such kind of protein to use as an example, which we can conveniently do with hpacker in-built functions
hpacker = HPacker('T0950.pdb')
hpacker.remove_sidechains_for_res_ids(hpacker.get_res_ids()[:10])
hpacker.write_pdb('T0950_partial.pdb')

By default, `.refine_sidechains()` will add the side-chain of any residue that is missing it, and will further re-pack all side-chains for residues whose beta-Carbon is within 10 Angstroms of any missing side-chains' beta-Carbons. The proximity threshold can be changed by setting the `proximity_cutoff_for_refinement` parameter.

In [30]:
hpacker = HPacker('T0950_partial.pdb')
hpacker.reconstruct_sidechains(num_refinement_iterations = 5, proximity_cutoff_for_refinement = 10.0)
hpacker.write_pdb('T0950_partial_reconstructed_with_proximity_cutoff.pdb')

The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


The cell above uses the amino-acid identities (resnames) present in the original structure to determine which side-chains to add.
If the input structure does not contain any amino-acid information for those residues, or if you want to specify them anew, you can do so by passing in a dictionary of the form `{res_id: resname}` to the `res_id_to_resname` parameter, where `res_id` is a tuple of the form `(chain, resnum, icode)`.

In [31]:
hpacker = HPacker('T0950_partial.pdb')
mutations = {res_id: 'TRP' for res_id in hpacker.get_res_ids()[:10]} # mutating to TRP
hpacker.reconstruct_sidechains(num_refinement_iterations = 5, proximity_cutoff_for_refinement = 10.0, res_id_to_resname=mutations)
hpacker.write_pdb('T0950_partial_reconstructed_with_mutations.pdb')

In alternative to selecting surrounding residues to re-pack via `proximity_cutoff_for_refinement`, we can also specify a list of residues via the `res_ids_to_refine` parameter. Note that these residues are re-packed *in addition* to the ones that were originally missing the side-chains.

In [32]:
hpacker = HPacker('T0950_partial.pdb')
hpacker.reconstruct_sidechains(num_refinement_iterations = 5, res_ids_to_refine=hpacker.get_res_ids()[10:20])
hpacker.write_pdb('T0950_partial_reconstructed_with_res_ids_to_refine.pdb')

##### If the input structure has all side-chains but you want to easily re-pack or apply mutations...

First, if you only want to re-pack a subset of residues using the refinement model, you can use the `.refine_sidechains()` method described at the end of the previous section.

If instead you want to apply some mutations in a convenient way, you can simply pass them into the `res_id_to_resname`parameter of `.reconstruct_sidechains()`. This is effectively equivalent to passing the structure without side-chains at the residues you want to mutate, just like a few cells above:

In [33]:
hpacker = HPacker('T0950.pdb')
mutations = {res_id: 'TRP' for res_id in hpacker.get_res_ids()[:10]} # mutating to TRP
hpacker.reconstruct_sidechains(num_refinement_iterations = 5, proximity_cutoff_for_refinement = 10.0, res_id_to_resname=mutations)
hpacker.write_pdb('T0950_reconstructed_with_mutations.pdb') # --> equivalent to what was saved in 'T0950_partial_reconstructed_with_mutations.pdb'

Setting `reconstruct_all_sidechains` to `True` will make HPacker reconstruct all side-chains, accounting for the mutations specified in `res_id_to_resname`.

In [34]:
hpacker = HPacker('T0950.pdb')
mutations = {res_id: 'TRP' for res_id in hpacker.get_res_ids()[:10]} # mutating to TRP
hpacker.reconstruct_sidechains(num_refinement_iterations = 5, proximity_cutoff_for_refinement = 10.0, res_id_to_resname=mutations, reconstruct_all_sidechains=True)
hpacker.write_pdb('T0950_reconstructed_fully_with_mutations.pdb')

Reconstructing all sidechains from scratch


# Slow Start - the HPacker object

#### Constructor
Each HPacker object is utilized to reconstruct the side-chains of a single protein structure.
Indeed, the constructor takes as required inputs a single PDB file, and a tuple of two paths pointing to the "Initial Guess" and the "Refinement" model directories. There are also several keyword arguments that govern pre-processing of the input structure, such as filtering out hydrogens and waters, and to remove sidechains.

#### Internal Representation
HPacker uses BioPython's PDB module to store and manipulate structures. HPacker stores three versions of the structure, as ```Bio.PDB.Structure``` objects:
1. ```self.structure``` stores the *current* version of the structure, as it's being manipulated or reconstructed. Importantly, the backbone is never changed from the input.
2. ```self.original_structure``` always stores the original structure passed to the constructor, after filtering but *before* removing side-chains, if the kwarg ```remove_sidechains``` is set to ```True```.
3. ```self.copy_structure``` is a dummy structure used as a helper by the internal processes.

Residue sites are uniquely identified by residue IDs (```res_id```), each consisting of the Tuple (chain, resnum, icode).
Individual Residue objects and amino-acid types (```resname```) can be accessed using the ```res_id``` within HPacker methods.

#### Useful methods
Below is a list of *some* of the Hpacker methods that can be used to conveniently manipulate the internal representation of the protein structure
- `get_res_ids()`
- `get_residue(res_id)`
- `get_resname(res_id)`
- `update_resnames(res_id_to_resnames)`
- `write_pdb(outputpath)`